# Import Relevant Data

In [1]:
import pandas as pd
import ast
import numpy as np
import math
import time
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt 
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [2]:
df_cryp=pd.read_csv('https://raw.githubusercontent.com/SunYutongAmber/SignatureWork/main/Data/gas_rarity_usd_merged.csv')
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,skin_tone_color,img_url,Rarity_Score_1_rarest,Rarity_Score_2_Average,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date,eth_usd_price
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,12.040656,9.048897,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,74.786090,26.889225,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,7.460154,6.064456,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,1.478706,1.478706,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...,7.585070,5.654169,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...,5.427700,3.061263,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,4.715707,4.285346,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,18.331532,8.951758,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,19.315489,7.224599,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98


# Volume of Active Market Wallet & Daily Change Rate

In [4]:
df_group=df_cryp.groupby('date_x').count()
active_wallet=np.array(df_group['Unnamed: 0'])*2
df_wallet=DataFrame({"active_wallet": active_wallet,
                     "date":df_group.index})
df_wallet

,active_wallet,date
0,8,2018-01-18
1,12,2018-01-19
2,38,2018-01-20
3,4,2018-01-22
4,12,2018-01-23
...,...,...
1212,4,2022-07-22
1213,4,2022-07-23
1214,12,2022-07-25
1215,6,2022-07-26


In [5]:
df_wallet['active_wallet_shift']=df_wallet['active_wallet'].shift(1)
df_wallet['active_wallet_shift'][0]=3
df_wallet

<ipython-input-5-51ec93477ae8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wallet['active_wallet_shift'][0]=3


,active_wallet,date,active_wallet_shift
0,8,2018-01-18,3.0
1,12,2018-01-19,8.0
2,38,2018-01-20,12.0
3,4,2018-01-22,38.0
4,12,2018-01-23,4.0
...,...,...,...
1212,4,2022-07-22,4.0
1213,4,2022-07-23,4.0
1214,12,2022-07-25,4.0
1215,6,2022-07-26,12.0


In [6]:
wallet_change=(np.array(df_wallet['active_wallet'])-np.array(df_wallet['active_wallet_shift']))/np.array(df_wallet['active_wallet_shift'])
df_wallet['wallet_change']=wallet_change
df_wallet['wallet_change']

0       1.666667
1       0.500000
2       2.166667
3      -0.894737
4       2.000000
          ...   
1212    0.000000
1213    0.000000
1214    2.000000
1215   -0.500000
1216   -0.333333
Name: wallet_change, Length: 1217, dtype: float64

In [7]:
df_cryp=df_cryp.merge(df_wallet, left_on='date_x', right_on='date')

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [10]:
df_cryp.to_csv('wallet_rarity_usd_gas_merged.csv')
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57,12,2018-01-19,8.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333


# Volume of Cryptopunks Sales Daily Change Rate

In [14]:
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57,12,2018-01-19,8.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333


In [15]:
sales=df_cryp.groupby('timestamp')['eth_price'].sum()
sales

timestamp
1.516234e+09      0.58
1.516320e+09      0.79
1.516406e+09      2.46
1.516579e+09      0.80
1.516666e+09      1.24
                 ...  
1.658448e+09    177.00
1.658534e+09    198.00
1.658707e+09    410.40
1.658794e+09    257.90
1.658880e+09    174.64
Name: eth_price, Length: 1217, dtype: float64

In [16]:
df_sales=DataFrame({'daily_sales':sales,
                    'date_z':df_group.index})
df_sales

,daily_sales,date_z
timestamp,,
1.516234e+09,0.58,2018-01-18
1.516320e+09,0.79,2018-01-19
1.516406e+09,2.46,2018-01-20
1.516579e+09,0.80,2018-01-22
1.516666e+09,1.24,2018-01-23
...,...,...
1.658448e+09,177.00,2022-07-22
1.658534e+09,198.00,2022-07-23
1.658707e+09,410.40,2022-07-25


In [17]:
df_sales['sales_shift']=df_sales['daily_sales'].shift(1)
df_sales['sales_shift'][0]=0.42
change_sales=(np.array(df_sales['daily_sales'])-np.array(df_sales['sales_shift']))/np.array(df_sales['sales_shift'])
change_sales

<ipython-input-17-c0bd02db3cfe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['sales_shift'][0]=0.42
<ipython-input-17-c0bd02db3cfe>:3: RuntimeWarning: divide by zero encountered in true_divide
  change_sales=(np.array(df_sales['daily_sales'])-np.array(df_sales['sales_shift']))/np.array(df_sales['sales_shift'])


array([ 0.38095238,  0.36206897,  2.11392405, ...,  1.07272727,
       -0.37158869, -0.32283831])

In [18]:
df_sales['daily_sales_change']=change_sales
df_sales

,daily_sales,date_z,sales_shift,daily_sales_change
timestamp,,,,
1.516234e+09,0.58,2018-01-18,0.42,0.380952
1.516320e+09,0.79,2018-01-19,0.58,0.362069
1.516406e+09,2.46,2018-01-20,0.79,2.113924
1.516579e+09,0.80,2018-01-22,2.46,-0.674797
1.516666e+09,1.24,2018-01-23,0.80,0.550000
...,...,...,...,...
1.658448e+09,177.00,2022-07-22,168.68,0.049324
1.658534e+09,198.00,2022-07-23,177.00,0.118644
1.658707e+09,410.40,2022-07-25,198.00,1.072727


In [22]:
df_cryp=df_cryp.merge(df_sales,left_on='timestamp',right_on='timestamp')

In [27]:
df_cryp.to_csv('sales_wallet_rarity_gas_usd.csv')

In [25]:
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change,daily_sales,date_z,sales_shift,daily_sales_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,2018-1-19,979.57,12,2018-01-19,8.0,0.500000,0.79,2018-01-19,0.58,0.362069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333,174.64,2022-07-27,257.90,-0.322838
